In [2]:
import graphlab

# Load a common image analysis dataset

In [4]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

## Exploring the image data

In [5]:
# graphlab.canvas.set_target('ipynb')

In [6]:
image_train['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:9005/index.html
Opening Canvas in default web browser.


# Train a classifier on the raw image pixels

In [8]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train, target='label',
                                              features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1895

...

| 6         | 14       | 1.000000  | 8.415554     | 0.417942          | 0.463636            |

| 10        | 18       | 1.000000  | 10.732099    | 0.484433          | 0.536364            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## Make a prediction with the simple model based on raw pixels

In [9]:
image_test[0:3]['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:9005/index.html
Opening Canvas in default web browser.


In [10]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [11]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['dog', 'dog', 'bird']

# Evaluating raw pixel model on test data

In [12]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.4655, 'auc': 0.6984569583333347, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     cat      |       bird      |  238  |
 |  automobile  |       dog       |  186  |
 |  automobile  |       bird      |   99  |
 |     bird     |       bird      |  496  |
 |     cat      |       cat       |  259  |
 |     cat      |       dog       |  375  |
 |     dog      |       dog       |  506  |
 |     dog      |    automobile   |   90  |
 |     bird     |       cat       |  133  |
 |     cat      |    automobile   |  128  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.4619860037262565, 'log_loss': 1.2538531329500668, 'precision': 0.

# Can we improve the model using deep features

In [13]:
len(image_train)

2005

In [15]:
len(image_test)

4000

In [16]:
deep_learning_model = graphlab.load_model('imagenet_model')
image_train['deep_features'] = deep_learning_model.extract_features(image_train)

IOError: C:\Users\marco\Desktop\Machine Learning\Week 6\imagenet_model is not a valid file name.

In [17]:
image_train

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features lets train a classifier

In [19]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 9         | 16       | 0.250000  | 12.164402    | 0.846837          | 0.750000            |

| 10        | 17       | 0.250000  | 13.107312    | 0.867747          | 0.750000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## Apply the deep features model to first few images of test set

In [25]:
image_test[0:10]['label']

dtype: str
Rows: 10
['cat', 'automobile', 'cat', 'automobile', 'dog', 'dog', 'dog', 'bird', 'dog', 'dog']

In [26]:
deep_features_model.predict(image_test[0:10])

dtype: str
Rows: 10
['cat', 'automobile', 'cat', 'automobile', 'dog', 'dog', 'dog', 'bird', 'dog', 'cat']

## Compute test_data accuracy of deep_features_model

In [27]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.7845, 'auc': 0.9406580416666699, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     dog      |    automobile   |   19  |
 |     bird     |       bird      |  775  |
 |  automobile  |       bird      |   19  |
 |     dog      |       bird      |   38  |
 |     cat      |    automobile   |   40  |
 |     cat      |       cat       |  637  |
 |     bird     |       cat       |  121  |
 |     cat      |       dog       |  258  |
 |     dog      |       dog       |  765  |
 |  automobile  |    automobile   |  961  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7838434674275738, 'log_loss': 0.5620832194302905, 'precision': 0.